<a href="https://colab.research.google.com/github/robbi621/skripsi/blob/main/testing_skripsi_saham_FIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px

In [2]:
!wget -O UNTR.csv https://raw.githubusercontent.com/robbi621/skripsi/main/UNTR.JK.csv

--2022-10-29 08:55:07--  https://raw.githubusercontent.com/robbi621/skripsi/main/UNTR.JK.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254503 (249K) [text/plain]
Saving to: ‘UNTR.csv’

UNTR.csv            100%[===================>] 248.54K  --.-KB/s    in 0.03s   

2022-10-29 08:55:07 (7.91 MB/s) - ‘UNTR.csv’ saved [254503/254503]



In [3]:
read_df = pd.read_csv("UNTR.csv")
read_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-11,16584.625000,16632.695313,15863.554688,16584.625000,10509.820313,11185970.0
1,2010-01-12,17209.552734,17257.625000,16536.552734,17209.552734,10905.845703,6474226.0
2,2010-01-13,17065.337891,17113.410156,16873.052734,17065.337891,10814.453125,3540568.0
3,2010-01-14,17594.123047,17594.123047,17257.625000,17594.123047,11149.550781,5474151.0
4,2010-01-15,17738.337891,17834.480469,17642.195313,17738.337891,11240.942383,4166720.0


In [4]:
# Rename columns
read_df.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close"}, inplace= True)
read_df.head()

,date,open,high,low,close,Adj Close,Volume
0,2010-01-11,16584.625000,16632.695313,15863.554688,16584.625000,10509.820313,11185970.0
1,2010-01-12,17209.552734,17257.625000,16536.552734,17209.552734,10905.845703,6474226.0
2,2010-01-13,17065.337891,17113.410156,16873.052734,17065.337891,10814.453125,3540568.0
3,2010-01-14,17594.123047,17594.123047,17257.625000,17594.123047,11149.550781,5474151.0
4,2010-01-15,17738.337891,17834.480469,17642.195313,17738.337891,11240.942383,4166720.0


In [5]:
# Checking null value
read_df.isnull().sum()

date         0
open         1
high         1
low          1
close        1
Adj Close    1
Volume       1
dtype: int64

In [6]:
# Checking na value
read_df.isna().any()

date         False
open          True
high          True
low           True
close         True
Adj Close     True
Volume        True
dtype: bool

In [7]:
read_df.dropna(inplace=True)
read_df.isna().any()

date         False
open         False
high         False
low          False
close        False
Adj Close    False
Volume       False
dtype: bool

In [8]:
# Checking Data type of each column
print("Date column data type: ", type(read_df['date'][0]))
print("Open column data type: ", type(read_df['open'][0]))
print("Close column data type: ", type(read_df['close'][0]))
print("High column data type: ", type(read_df['high'][0]))
print("Low column data type: ", type(read_df['low'][0]))

Date column data type:  <class 'str'>
Open column data type:  <class 'numpy.float64'>
Close column data type:  <class 'numpy.float64'>
High column data type:  <class 'numpy.float64'>
Low column data type:  <class 'numpy.float64'>


In [9]:
# convert date field from string to Date format and make it index
read_df['date'] = pd.to_datetime(read_df.date)
read_df.head()

,date,open,high,low,close,Adj Close,Volume
0,2010-01-11,16584.625000,16632.695313,15863.554688,16584.625000,10509.820313,11185970.0
1,2010-01-12,17209.552734,17257.625000,16536.552734,17209.552734,10905.845703,6474226.0
2,2010-01-13,17065.337891,17113.410156,16873.052734,17065.337891,10814.453125,3540568.0
3,2010-01-14,17594.123047,17594.123047,17257.625000,17594.123047,11149.550781,5474151.0
4,2010-01-15,17738.337891,17834.480469,17642.195313,17738.337891,11240.942383,4166720.0


In [10]:
read_df.sort_values(by='date', inplace=True)
read_df.head()

,date,open,high,low,close,Adj Close,Volume
0,2010-01-11,16584.625000,16632.695313,15863.554688,16584.625000,10509.820313,11185970.0
1,2010-01-12,17209.552734,17257.625000,16536.552734,17209.552734,10905.845703,6474226.0
2,2010-01-13,17065.337891,17113.410156,16873.052734,17065.337891,10814.453125,3540568.0
3,2010-01-14,17594.123047,17594.123047,17257.625000,17594.123047,11149.550781,5474151.0
4,2010-01-15,17738.337891,17834.480469,17642.195313,17738.337891,11240.942383,4166720.0


In [11]:
read_df.shape

(3034, 7)

In [12]:
print("Starting date: ",read_df.iloc[0][0])
print("Ending date: ", read_df.iloc[-1][0])
print("Duration: ", read_df.iloc[-1][0]-read_df.iloc[0][0])

Starting date:  2010-01-11 00:00:00
Ending date:  2022-03-31 00:00:00
Duration:  4462 days 00:00:00


In [13]:
monthvise= read_df.groupby(read_df['date'].dt.strftime('%B'))[['open','close']].mean().sort_values(by='close')
monthvise.head()

,open,close
date,,
June,22055.065164,22053.165164
September,22254.548111,22278.663598
May,22336.518224,22303.673035
February,22500.728750,22495.709065
July,22505.153628,22500.970361


In [14]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock actual, open and close price')
fig.show()

In [15]:
read_df.groupby(read_df['date'].dt.strftime('%B'))['low'].min()

date
April        14600.000000
August       13650.000000
December     13925.000000
February     14373.341797
January      15625.000000
July         14600.000000
June         13375.000000
March        12000.000000
May          12550.000000
November     16050.000000
October      16400.000000
September    15225.000000
Name: low, dtype: float64

In [16]:
monthvise_high= read_df.groupby(read_df['date'].dt.strftime('%B'))['high'].max()
monthvise_low= read_df.groupby(read_df['date'].dt.strftime('%B'))['low'].min()

In [17]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [18]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(read_df, x=read_df.date, y=[read_df['open'], read_df['close'], 
                                          read_df['high'], read_df['low']],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [19]:
closedf = read_df[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (3034, 2)


In [20]:
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [21]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(3034, 1)


In [22]:
training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (2123, 1)
test_data:  (911, 1)


In [23]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [24]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)

print("y_test", y_test.shape)

X_train:  (2107, 15)
y_train:  (2107,)
X_test:  (895, 15)
y_test (895,)


In [25]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)
predict = regressor.predict(X_test)
print(predict.shape)

(895,)


In [26]:
# Lets Do the prediction 

train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (2107, 1)
Test data prediction: (895, 1)


In [27]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [28]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))
print("-----------------------------------------")
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))
print("------------------------------------------")
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))
print("--------------------------------------")

Train data RMSE:  212.17274707867048
Train data MSE:  45017.27460290948
Test data MAE:  155.14807799400543
-------------------------------------------------------------------------------------
Test data RMSE:  609.4907342473805
Test data MSE:  371478.9551334109
Test data MAE:  457.7673698832738
-----------------------------------------
Train data explained variance regression score: 0.9986262792744002
Test data explained variance regression score: 0.9790363926926026
------------------------------------------
Train data R2 score: 0.998626155841276
Test data R2 score: 0.9790339263251915
--------------------------------------


In [29]:
look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (3034, 1)
Test predicted data:  (3034, 1)


In [30]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)

        yhat = regressor.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = regressor.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [31]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [32]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [33]:
knndf=closedf.tolist()
knndf.extend((np.array(lst_output).reshape(-1,1)).tolist())
knndf=scaler.inverse_transform(knndf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(knndf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()